# 单卡GPU 进行 ChatGLM3-6B模型 LORA 高效微调
本 Cookbook 将带领开发者使用 `AdvertiseGen` 对 ChatGLM3-6B 数据集进行 lora微调，使其具备专业的广告生成能力。

## 硬件需求
显存：24GB
显卡架构：安培架构（推荐）
内存：16GB

## 1. 准备数据集
我们使用 AdvertiseGen 数据集来进行微调。从 [Google Drive](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view?usp=sharing) 或者 [Tsinghua Cloud](https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1) 下载处理好的 AdvertiseGen 数据集，将解压后的 AdvertiseGen 目录放到本目录的 `/data/` 下, 例如。
> /media/zr/Data/Code/ChatGLM3/finetune_demo/data/AdvertiseGen

接着，运行本代码来切割数据集

In [2]:
import json
from typing import Union
from pathlib import Path

# 将路径转换为Path对象，并解析环境变量和用户路径
def _resolve_path(path: Union[str, Path]) -> Path: 
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    # 解析路径
    dir_name = _resolve_path(dir_name)
    # 检查路径是否为目录
    if not dir_name.is_dir():
        # 创建目录，如果有父目录，则创建父目录
        dir_name.mkdir(parents=True, exist_ok=False)


import json
from pathlib import Path
from typing import Union

def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):

    # 将AdvertiseGen数据集转换为fix格式
    def _convert(in_file: Path, out_file: Path):

        # 将一行的数据转换为fix格式的数据
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/AdvertiseGen', 'data/AdvertiseGen_fix')

## 2. 使用命令行开始微调,我们使用 lora 进行微调
接着，我们仅需要将配置好的参数以命令行的形式传参给程序，就可以使用命令行进行高效微调，这里将 `/media/zr/Data/Code/ChatGLM3/venv/bin/python3` 换成你的 python3 的绝对路径以保证正常运行。

- python3的绝对路径：/root/miniconda3/envs/chatglm/bin/python3
- chatglm3-6b的绝对路径：/root/models/chatglm3-6b

In [4]:
!CUDA_VISIBLE_DEVICES=0 /root/miniconda3/envs/chatglm/bin/python3 finetune_hf.py  data/AdvertiseGen_fix  /root/models/chatglm3-6b  configs/lora.yaml

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  2.18it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614
--> Model

--> model has 1.949696M params

Map (num_proc=8): 100%|███████| 114599/114599 [00:04<00:00, 24484.36 examples/s]
train_dataset: Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 114599
})
Map (num_proc=8): 100%|████████████| 1070/1070 [00:00<00:00, 2007.32 examples/s]
val_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
Map (num_proc=8): 100%|████████████| 1070/1070 [00:00<00:00, 1875.02 examples/s]
test_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
--> Sanity check
           '[gMASK]': 64790 -> -100
               'sop': 64792 -> -100
          '<|us

## 3. 使用微调的数据集进行推理
在完成微调任务之后，我们可以查看到 `output` 文件夹下多了很多个`checkpoint-*`的文件夹，这些文件夹代表了训练的轮数。
我们选择最后一轮的微调权重，并使用inference进行导入。

In [1]:
!ls output/

checkpoint-1000  checkpoint-2000  checkpoint-3000
checkpoint-1500  checkpoint-2500  checkpoint-500


In [2]:
!CUDA_VISIBLE_DEVICES=0  /root/miniconda3/envs/chatglm/bin/python3 inference_hf.py  /root/ChatGLM3/finetune_demo/output/checkpoint-3000/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:06<00:00,  1.11it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
套头式连衣裙，方便穿脱，适合各种身材的宝宝。网纱拼接，增加层次感，更显甜美。前片压褶百褶设计，增加裙子的层次感，增加裙子的立体感。袖口和领口木耳边点缀，凸显甜美可爱。下摆不规则压褶，显瘦显高。背部的拉链设计，方便穿脱，增加裙子的细节感。


## 4. 总结
到此位置，我们就完成了使用单张 GPU Lora 来微调 ChatGLM3-6B 模型，使其能生产出更好的广告。
在本章节中，你将会学会：
+ 如何使用模型进行 Lora 微调
+ 微调数据集的准备和对齐
+ 使用微调的模型进行推理